In [ ]:
# Source code : Bradford council data comparison of accident. 
# Date Written : 30-11-2023 - 05-12-2023
# Programmer : Anish Vattemkuzhyil Appukuttan
# Description : This creates a heatmap of Kent just like the way
# we created for Bradford using DBSCAN. But as there is no data , we will not be able to see the clusters. This
# can be used in the future.


In [2]:
import pandas as pd
import geopandas as gpd
from sklearn.cluster import DBSCAN
import folium
from folium.plugins import HeatMap
import warnings




df = pd.read_csv('C:/Users/44743/Desktop/Data Science/Bedford/bfd_crashes.csv')

# Converting the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude))

# Defining the location for Kent
kent_coords = [51.2787, 0.5217]  # Latitude and longitude for Kent

# Creating a folium map centered around Kent
map = folium.Map(location=kent_coords, zoom_start=10)

# Creating a HeatMap layer and add it to the base map
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in gdf.geometry]
HeatMap(heat_data).add_to(map)

# Applying DBSCAN to identify clusters
coords = gdf[['latitude', 'longitude']]
db = DBSCAN(eps=0.001, min_samples=10).fit(coords)  # Adjust eps and min_samples as needed
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels)) - (1 if -1 in cluster_labels else 0)
clusters = pd.Series([coords[cluster_labels == n] for n in range(num_clusters)])

# Add the cluster centers to the map
for cluster in clusters:
    if not cluster.empty:
        folium.Marker(
            location=[cluster.latitude.mean(), cluster.longitude.mean()],
            icon=folium.Icon(color='blue', icon='info-sign'),
            popup=f"Cluster Center\nCount: {len(cluster)}",
        ).add_to(map)

# Save the map to an HTML file
map.save('kent_accident_heatmap.html')
warnings.filterwarnings('ignore')

